In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import pandas as pd
import numpy as np

# Some useful utilities

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

def z_clip(xs, b):
    return [min(x, b) for x in xs]

def clip(xs, upper, lower):
    return [max(min(x, upper), lower) for x in xs]

def gaussian_mech_vec(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon, size=len(v))

def L2_clip(v, b):
    norm = np.linalg.norm(v, ord=2)
    
    if norm > b:
        return b * (v / norm)
    else:
        return v

def your_code_here():
    return 1

def test(msg, value, expected):
    if value == expected:
        print(f"{msg}: {value}, as expected")
    else:
        print(f"{msg}: OH NO! Got {value}, but expected {expected}.")

In [3]:
X = np.load('adult_processed_x.npy')
y = np.load('adult_processed_y.npy')

training_size = int(X.shape[0] * 0.8)

X_train = X[:training_size]
X_test = X[training_size:]

y_train = y[:training_size]
y_test = y[training_size:]

In [4]:
# Prediction: take a model (theta) and a single example (xi) and return its predicted label
def predict(theta, xi):
    label = np.sign(xi @ theta)
    return label

# The loss function measures how good our model is. The training goal is to minimize the loss.
# This is the logistic loss function.
def loss(theta, xi, yi):
    exponent = - yi * (xi.dot(theta))
    return np.log(1 + np.exp(exponent))

# This is the gradient of the logistic loss
# The gradient is a vector that indicates the rate of change of the loss in each direction
def gradient(theta, xi, yi):
    exponent = yi * (xi.dot(theta))
    return - (yi*xi) / (1+np.exp(exponent))

def accuracy(theta):
    return np.sum(predict(theta, X_test) == y_test)/X_test.shape[0]

# Simple gradient descent algorithm
def avg_grad(theta, X, y):
    grads = [gradient(theta, xi, yi) for xi, yi in zip(X, y)]
    return np.mean(grads, axis=0)

def gradient_descent(iterations):
    theta = np.zeros(X_train.shape[1])

    for i in range(iterations):
        theta = theta - avg_grad(theta, X_train, y_train)

    return theta

In [5]:
theta = gradient_descent(10)
accuracy(theta)

0.7787483414418399

----------------

## Question 1 (30 points)

Implement a function `dp_gradient_descent` that performs differentially private gradient descent by adding noise to the gradient at each iteration. Your function should take additional arguments $\epsilon$ and $\delta$, and should have an **overall** privacy cost of $(\epsilon, \delta)$-differential privacy. You may target *either* bounded or unbounded differential privacy.

**Note**: this is a major difference from the function defined in the notes, which bounds $\epsilon$ *per-iteration*. Your solution should bound the *total* privacy cost of training.

*Hint*: Use `gaussian_mech_vec`, defined above, to add noise.

*Hint*: Use advanced composition to bound the total privacy cost. Start with the total privacy cost of $k$-fold adaptive composition under advanced composition, then solve for $\epsilon_i$, the privacy cost per iteration. Use this result to set the per-iteration value of `epsilon`, and similar for `delta`.

In [6]:
# Calculate sum with an enforced L2 sensitivity of clipped X
def clipped_gradients_sum(theta, X, y, b):
    gradients = [gradient(theta, X_i, y_i) for X_i, y_i in zip(X,y)]
    return np.sum(gradients, axis=0)

def dp_gradient_descent(iterations, epsilon, delta):

    # Constants
    theta = np.zeros(X_train.shape[1])
    noisy_count_epsilon = epsilon * 0.25
    gradient_descent_epsilon = epsilon * 0.75
    epsilon_i = gradient_descent_epsilon / iterations
    delta_i = delta/iterations
    
    # Define a global sensitivity
    sensitivity = 5
    
    # Calculate noisy clipped data
    noisy_count = laplace_mech(X_train.shape[0], 1, noisy_count_epsilon)
    clipped_X = [L2_clip(X_i, sensitivity) for X_i in X_train]
    
    # Perform gradient descent
    for i in range(iterations):
        gradient_sum = clipped_gradients_sum(theta, clipped_X, y_train, sensitivity)
        noisy_gradient_sum = gaussian_mech_vec(gradient_sum, sensitivity, epsilon_i, delta_i)
        noisy_avg_gradient = noisy_gradient_sum/noisy_count
        theta = theta - noisy_avg_gradient
        
    return theta

delta = 1e-5

for epsilon in [0.01, 0.1, 1.0]:
    theta = dp_gradient_descent(10, epsilon, delta)
    acc = accuracy(theta)
    print(f"Epsilon = {epsilon}, final accuracy: {acc}")

Epsilon = 0.01, final accuracy: 0.7533171163202123
Epsilon = 0.1, final accuracy: 0.7749889429455993
Epsilon = 1.0, final accuracy: 0.7785272003538257


## Question 2 (10 points)

In 2-5 sentences, argue that your implementation of `dp_gradient_descent` satisfies $(\epsilon, \delta)$-differential privacy.

- The privacy cost of this algorithm would normally be ($k\epsilon + \epsilon, k\delta$): the gradient descent process satisfies ($\epsilon, \delta$)-differential privacy, and we make an additional query in order to determine the noisy count. To reduce this privacy cost, the algortihm uses a quarter of the privacy budget to determine the noisy count, and the larger rest to perform the gradient descent which itself uses a total of $\epsilon_i$ per iteration.

## Question 3 (40 points)

Implement a function `zcdp_gradient_descent` that performs differentially private gradient descent by adding noise to the gradient at each iteration. Your function should take an additional argument $\rho$, and should have an **overall** privacy cost of $\rho$-zero concentrated differential privacy. You will also have to implement `gaussian_mech_vec_zcdp`, the vector-valued gaussian mechanism for zCDP.

In [7]:
def gaussian_mech_zCDP_vec(v, sensitivity, rho):
    
    # Calculate normal dist scale
    sigma = np.sqrt((sensitivity**2) / (2*rho))
    return v + np.random.normal(scale=sigma)

def zcdp_gradient_descent(iterations, rho):
    
    # Constants
    theta = np.zeros(X_train.shape[1])
    rho_i = rho/iterations
    
    # Define a global sensitivity
    sensitivity = 5
    
    # Calculate noisy clipped data
    noisy_count = laplace_mech(X_train.shape[0], 1, rho)
    clipped_X = [L2_clip(X_i, sensitivity) for X_i in X_train]
    
    # Perform gradient descent
    for i in range(iterations):
        gradient_sum = clipped_gradients_sum(theta, clipped_X, y_train, sensitivity)
        noisy_gradient_sum = gaussian_mech_zCDP_vec(gradient_sum, sensitivity, rho_i)
        noisy_avg_gradient = noisy_gradient_sum/noisy_count
        theta = theta - noisy_avg_gradient
        
    return theta

for rho in [0.00001, 0.0001, 0.001]:
    theta = zcdp_gradient_descent(10, rho)
    acc = accuracy(theta)
    print(f"rho = {rho}, final accuracy: {acc}")

rho = 1e-05, final accuracy: 0.7585139318885449
rho = 0.0001, final accuracy: 0.7639318885448917
rho = 0.001, final accuracy: 0.7838345864661654


## Question 4 (20 points)

Implement a function `convert_zCDP_eps_delta` that converts a $\rho$-zCDP privacy bound to a $(\epsilon,\delta)$-differential privacy bound, given a target $\delta$.

In [8]:
# Implement: epsilon = rho + 2sqrt(rho*log(1/delta))
def convert_zCDP_eps_delta(rho, delta):
    return rho + (2 * np.sqrt(rho * np.log(1 / delta)))

delta = 1e-5

for rho in [0.00001, 0.0001, 0.001]:
    theta = zcdp_gradient_descent(10, rho)
    acc = accuracy(theta)
    epsilon = convert_zCDP_eps_delta(rho, delta)
    print(f"rho = {rho}, epsilon = {epsilon}, final accuracy: {acc}")

rho = 1e-05, epsilon = 0.021469660262893472, final accuracy: 0.24148606811145512
rho = 0.0001, epsilon = 0.06796140424415112, final accuracy: 0.794670499778859
rho = 0.001, epsilon = 0.21559660262893474, final accuracy: 0.7783060592658115
